In [2]:
import jax
import jax.numpy as jnp
from approx_post.distributions import approx, joint
from approx_post import losses, optimisers

In [3]:
def create_data(model, true_theta, noise_cov, num_samples, ndim, prngkey):
    mean = model(true_theta)
    samples = jax.random.multivariate_normal(key=prngkey, mean=mean, cov=noise_cov, shape=(num_samples,))
    return samples.reshape(num_samples, -1)

In [4]:
# First, let's define a model:
ndim = 1
model = lambda theta: theta**2
model_grad = jax.vmap(jax.vmap(jax.jacfwd(model), in_axes=0), in_axes=0)

In [5]:
# Create artificial data:
prngkey = jax.random.PRNGKey(10)
true_theta = jnp.array([2])
noise_cov = 1*jnp.identity(ndim)
num_samples = 1
data = create_data(model, true_theta, noise_cov, num_samples, ndim, prngkey)
print(f'True theta: \n {true_theta}')
print(f'True x = model(theta): \n {model(true_theta)}')
print(f'Observations x_obs = model(theta) + noise: \n {data}')

True theta: 
 [2]
True x = model(theta): 
 [4]
Observations x_obs = model(theta) + noise: 
 [[2.6554594]]


In [6]:
# Create Gaussian approximate distribution:
approx_dist = approx.Gaussian(ndim)

In [7]:
# Create Joint distribution from forward model:
prior_mean = jnp.zeros(ndim)
prior_cov = 0.1*jnp.identity(ndim)
joint_dist = joint.ModelPlusGaussian(model, noise_cov, prior_mean, prior_cov, model_grad)

In [8]:
approx_dist = approx.Gaussian(ndim)
prngkey = jax.random.PRNGKey(12)
loss = losses.ELBO(joint_dist, use_reparameterisation=True)
optimiser = optimisers.Adam()
optimiser.fit(approx_dist, loss, data, prngkey, verbose = True, max_iter=1000, num_samples=1000)

Loss = 7.13171911239624, Params = Jaxtainer({'mean': DeviceArray([-0.1], dtype=float32), 'log_chol_diag': DeviceArray([-0.1], dtype=float32)})
Loss = 6.170280456542969, Params = Jaxtainer({'mean': DeviceArray([-0.09584227], dtype=float32), 'log_chol_diag': DeviceArray([-0.19757421], dtype=float32)})
Loss = 5.504497528076172, Params = Jaxtainer({'mean': DeviceArray([-0.0578491], dtype=float32), 'log_chol_diag': DeviceArray([-0.29152155], dtype=float32)})
Loss = 5.034226894378662, Params = Jaxtainer({'mean': DeviceArray([-0.01444767], dtype=float32), 'log_chol_diag': DeviceArray([-0.3809414], dtype=float32)})
Loss = 4.718996524810791, Params = Jaxtainer({'mean': DeviceArray([0.01727934], dtype=float32), 'log_chol_diag': DeviceArray([-0.46525422], dtype=float32)})
Loss = 4.511056423187256, Params = Jaxtainer({'mean': DeviceArray([0.02946038], dtype=float32), 'log_chol_diag': DeviceArray([-0.544131], dtype=float32)})
Loss = 4.36901330947876, Params = Jaxtainer({'mean': DeviceArray([0.02451

KeyboardInterrupt: 

In [ ]:
approx_dist.cov()

In [ ]:
approx_dist = approx.Gaussian(ndim)
prngkey = jax.random.PRNGKey(12)
loss = losses.ForwardKL(joint_dist, use_reparameterisation=True)
optimiser = optimisers.Adam()
optimiser.fit(approx_dist, loss, data, prngkey, verbose = True, max_iter=1000, num_samples=1000)

In [ ]:
jnp.array(100).item()